<a href="https://colab.research.google.com/github/tasleema17/FINAL_PROJECT_22034202/blob/main/FloodPredictionCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.Import Libraries and Packages**

Load the essential libraries needed for data processing, visualization, and model building.

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

**2.Load and Preprocess Dataset**

Import the dataset, clean it by handling missing values, outliers, and inconsistencies to ensure it’s ready for analysis

In [76]:
# Load data flies
flood_data = pd.read_csv(r"/content/IndiaFloodInventory.csv")
rainfall_data = pd.read_csv(r'/content/Sub_Division_IMD_2017.csv')

In [77]:
# Display the first few rows of each dataset
print(flood_data.head())
print(rainfall_data.head())

   Unnamed: 0                   UEI        Start Date          End Date  \
0         563  UEI-IMD-FL-1967-0001  02/07/1967 00:00  08/07/1967 00:00   
1         564  UEI-IMD-FL-1967-0002  22/07/1967 00:00  28/07/1967 00:00   
2         565  UEI-IMD-FL-1967-0003  01/08/1967 00:00  30/08/1967 00:00   
3         566  UEI-IMD-FL-1967-0004  08/09/1967 00:00  09/09/1967 00:00   
4         567  UEI-IMD-FL-1968-0001  22/06/1968 00:00  28/06/1968 00:00   

   Duration(Days) Main Cause  \
0             7.0      flood   
1             7.0      flood   
2            30.0      flood   
3             2.0      flood   
4             7.0      flood   

                                           Districts  \
0                                                NaN   
1                                                NaN   
2                                                NaN   
3  Bhadrak, Dhenkanal, Jajapur, Subarnapur, Nuapa...   
4                                                NaN   

                   

In [78]:
#FLOOD DATA

# Convert date columns to datetime format
flood_data['Start Date'] = pd.to_datetime(flood_data['Start Date'], format='%d/%m/%Y %H:%M')
flood_data['End Date'] = pd.to_datetime(flood_data['End Date'], format='%d/%m/%Y %H:%M')
# Extract date features
flood_data['Year'] = flood_data['Start Date'].dt.year
flood_data['Month'] = flood_data['Start Date'].dt.month
flood_data['Day'] = flood_data['Start Date'].dt.day
flood_data['Season'] = flood_data['Start Date'].dt.month % 12 // 3 + 1  # 1: Winter, 2: Spring, 3: Summer, 4: Monsoon
flood_data['Day of Week'] = flood_data['Start Date'].dt.dayofweek

#RAINFALL DATA

# Melt rainfall_data to get a 'MONTH' and 'RAIN' column
rainfall_data_melted = rainfall_data.melt(
    id_vars=['SUBDIVISION', 'YEAR'],
    value_vars=['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'],
    var_name='MONTH',
    value_name='RAIN'
)

# Convert month abbreviations to numeric values (1 for January, 2 for February, etc.)
month_map = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}
rainfall_data_melted['MONTH'] = rainfall_data_melted['MONTH'].map(month_map)

# Ensure 'YEAR' is of type int
rainfall_data_melted['YEAR'] = rainfall_data_melted['YEAR'].astype(int)



In [79]:
#MERGE BOTH DATASETS
# Now merge with flood_data on 'Year' and 'Month'
merged_data = flood_data.merge(
    rainfall_data_melted,
    left_on=['Year', 'Month'],
    right_on=['YEAR', 'MONTH'],
    how='left'
)

# Fill missing rainfall data with 0
merged_data['RAIN'] = merged_data['RAIN'].fillna(0)
# Drop unnecessary columns after merging
merged_data = merged_data.drop(columns=['YEAR', 'UEI', 'Event Source', 'Year', 'Month', 'Day', 'Districts', 'Extent of damage ','Unnamed: 0'])

In [80]:
# Ensure the 'State' column is of type string
merged_data['State'] = merged_data['State'].astype(str)

# Ensure that NaN values in 'State' are replaced with an empty string, then convert to string
merged_data['State'] = merged_data['State'].replace(pd.NA, '').astype(str)

# Split the 'State' column by commas and explode into separate rows
merged_data['State'] = merged_data['State'].str.split(', ')
merged_data_exploded = merged_data.explode('State')

# Remove any rows where 'State' is empty (after handling NaN values)
merged_data_exploded = merged_data_exploded[merged_data_exploded['State'] != '']

# Clean the 'State' column to remove extraneous characters
merged_data_exploded['State'] = merged_data_exploded['State'].str.replace(
    r"[^a-zA-Z,& ]", "", regex=True
)
# Check the shape of the DataFrame
rows, columns = merged_data_exploded.shape
print(f"The DataFrame has {rows} rows and {columns} columns.")


The DataFrame has 161268 rows and 10 columns.


In [81]:
# Drop duplicate rows
merged_data_exploded = merged_data_exploded.drop_duplicates()

# Handle missing values using .loc
# For 'Main Cause', fill with 'Unknown'
merged_data_exploded.loc[:, 'Main Cause'] = merged_data_exploded['Main Cause'].fillna('Unknown')

# For 'SUBDIVISION', fill with 'Unknown'
merged_data_exploded.loc[:, 'SUBDIVISION'] = merged_data_exploded['SUBDIVISION'].fillna('Unknown')

# For 'RAIN', fill with the median value
median_rain = merged_data_exploded['RAIN'].median()
merged_data_exploded.loc[:, 'RAIN'] = merged_data_exploded['RAIN'].fillna(median_rain)

# Assuming 'merged_data_exploded' is your DataFrame
merged_data_exploded = merged_data_exploded.dropna(subset=['MONTH'])

# Verify the data cleaning
print("Missing values after cleaning:\n", merged_data_exploded.isnull().sum())
print(f"\nNumber of rows and columns after cleaning: {merged_data_exploded.shape}")


Missing values after cleaning:
 Start Date        0
End Date          0
Duration(Days)    0
Main Cause        0
State             0
Season            0
Day of Week       0
SUBDIVISION       0
MONTH             0
RAIN              0
dtype: int64

Number of rows and columns after cleaning: (157248, 10)


In [82]:
# Replace 'floods' and 'heavy floods' with 'flood' in "Main Cause"
merged_data_exploded['Main Cause'] = merged_data_exploded['Main Cause'].str.lower().replace(
    {'floods': 'flood', 'heavy floods': 'flood'}, regex=True
)

# Assuming your dataframe is named 'merged_data_exploded'
def categorize_cause(cause):
    # Convert to lowercase for uniform comparison
    cause_lower = cause.lower()

    # Category 1: Any term implying "flood" or "flash flood"
    if "flood" in cause_lower:
        return "flood"
    # Category 2: Descriptions mentioning "heavy rains," "landslide," or "cloud burst"
    elif any(term in cause_lower for term in ["heavy rains", "landslide", "cloud burst", "heavy rain"]):
        return "no flood"
    # Default to "no flood" if it doesn't match any criteria
    else:
        return "no flood"

# Apply the categorization function to the 'Main Cause' column
merged_data_exploded['Flood Occurance'] = merged_data_exploded['Main Cause'].apply(categorize_cause)
# Replace 'flood' with 1 and 'no flood' with 0 in the 'Flood Occurrence' column
merged_data_exploded['Flood Occurance'] = merged_data_exploded['Flood Occurance'].replace({'flood': 1, 'no flood': 0})

merged_data_exploded = merged_data_exploded.drop(columns=['Main Cause'])

# Count the occurrences of 'flood' and 'no flood' in the 'Flood Occurance' column
flood_counts = merged_data_exploded['Flood Occurance'].value_counts()

# Display the result
print(flood_counts)

Flood Occurance
0    121896
1     35352
Name: count, dtype: int64


<ipython-input-82-dce49864dc12>:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_data_exploded['Flood Occurance'] = merged_data_exploded['Flood Occurance'].replace({'flood': 1, 'no flood': 0})


In [83]:
# Define corrections dictionary to standardize state names
state_corrections = {
    "Jammu & Kashmir": "Jammu And Kashmir",
    "Daman & Diu": "Daman And Diu",
    "Maharastra": "Maharashtra",
    "Madras": "Tamil Nadu",
    "Tamilnadu": "Tamil Nadu",
    "Chennai": "Tamil Nadu",
    "Daman And Diu": "Daman,Diu",
    "Parts Of Maharastra": "Maharashtra",
    "East Rajasthan": "Rajasthan"

}
# Clean the 'State' column: remove leading/trailing spaces, apply title case, and apply corrections
merged_data_exploded['State'] = (
    merged_data_exploded['State']
    .str.strip()
    .str.title()
    .replace(state_corrections)
)

In [84]:
print(merged_data_exploded.head(157248))

       Start Date   End Date  Duration(Days)        State  Season  \
0      1967-07-02 1967-07-08             7.0        Assam     3.0   
1      1967-07-02 1967-07-08             7.0        Assam     3.0   
2      1967-07-02 1967-07-08             7.0        Assam     3.0   
3      1967-07-02 1967-07-08             7.0        Assam     3.0   
4      1967-07-02 1967-07-08             7.0        Assam     3.0   
...           ...        ...             ...          ...     ...   
154653 2017-08-08 2017-08-14             7.0  West Bengal     3.0   
154654 2017-08-08 2017-08-14             7.0  West Bengal     3.0   
154655 2017-08-08 2017-08-14             7.0  West Bengal     3.0   
154656 2017-08-08 2017-08-14             7.0  West Bengal     3.0   
154657 2017-08-08 2017-08-14             7.0  West Bengal     3.0   

        Day of Week                         SUBDIVISION  MONTH   RAIN  \
0               6.0           Andaman & Nicobar Islands    7.0  537.8   
1               6.0      

In [85]:
print(merged_data_exploded.isnull().sum())

Start Date         0
End Date           0
Duration(Days)     0
State              0
Season             0
Day of Week        0
SUBDIVISION        0
MONTH              0
RAIN               0
Flood Occurance    0
dtype: int64


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd

# Step 1: Separate features (X) and target (y)
X = merged_data_exploded.drop(columns=['Flood Occurance'])
y = merged_data_exploded['Flood Occurance']

# Step 2: Apply Random UnderSampling
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)

# Step 3: Combine X_resampled and y_resampled to create the resampled dataset
merged_data_resampled = pd.concat([X_resampled, y_resampled], axis=1)

# Step 4: Verify the balanced distribution of 'Flood Occurance'
print("Balanced Class Distribution (After Undersampling):\n", merged_data_resampled['Flood Occurance'].value_counts())

# Optionally, update merged_data_exploded with the resampled data
merged_data_exploded = merged_data_resampled


Balanced Class Distribution (After Undersampling):
 Flood Occurance
0    35352
1    35352
Name: count, dtype: int64


In [ ]:
# Count the occurrences of 'flood' and 'no flood' in the 'Flood Occurance' column
flood_counts = merged_data_exploded['Flood Occurance'].value_counts()

# Display the result
print(flood_counts)

In [ ]:
# Check the shape of the DataFrame
rows, columns = merged_data_exploded.shape
print(f"The DataFrame has {rows} rows and {columns} columns.")


INSIGHTS

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Rainfall Comparison by Subdivision
plt.figure(figsize=(10, 6))
sns.barplot(data=merged_data_exploded, x='SUBDIVISION', y='RAIN', palette='viridis')
plt.xticks(rotation=45, ha='right')
plt.title("Rainfall by Subdivision")
plt.ylabel("Rainfall (mm)")
plt.xlabel("Subdivision")
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Clean the 'State' column: remove leading/trailing spaces and make all entries lowercase
merged_data_exploded['State'] = merged_data_exploded['State'].str.strip().str.title()

# Group by 'State' and calculate the average rainfall per state
rainfall_per_state = merged_data_exploded.groupby('State')['RAIN'].mean().reset_index()

# Create the bar plot
plt.figure(figsize=(12, 6))  # Increase the figure size to make more room
sns.barplot(data=rainfall_per_state, x='State', y='RAIN', palette='muted', ci=None)

# Add title and labels
plt.title("Rainfall Summary per State")
plt.xlabel("State")
plt.ylabel("Average Rainfall (mm)")

# Rotate x-axis labels to prevent overlap
plt.xticks(rotation=90)

# Adjust layout to ensure the labels fit nicely
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# 4. Top 10 Rainiest States During Floods
top_states = merged_data_exploded.groupby('State')['RAIN'].mean().nlargest(10).reset_index()
plt.figure(figsize=(10, 6))
sns.barplot(data=top_states, x='State', y='RAIN', palette='magma')
plt.title("Top 10 Rainiest States During Floods")
plt.xlabel("State")
plt.ylabel("Average Rainfall (mm)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

MODEL EVALUATION

---



---



In [ ]:
print(merged_data_exploded.head())


In [ ]:
#random Forest

# Select features and target variable
X = merged_data_exploded.drop(columns=['Flood Occurance', 'Start Date', 'End Date'])  # Drop unnecessary columns
y = merged_data_exploded['Flood Occurance']

#Identify numeric columns
numeric_cols = merged_data_exploded.select_dtypes(include='number').columns

#Fill missing values in numeric columns with the mean of each column
merged_data_exploded[numeric_cols] = merged_data_exploded[numeric_cols].fillna(merged_data_exploded[numeric_cols].mean())

# Alternatively, fill missing values for categorical columns if necessary
merged_data_exploded = merged_data_exploded.fillna({'State': 'Unknown', 'SUBDIVISION': 'Unknown'})

# preprocessing steps
# Convert categorical variables to dummy/indicator variables
#merged_data_exploded = pd.get_dummies(merged_data_exploded, columns=['State', 'SUBDIVISION'], drop_first=True)


# Scale numerical features if necessary
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X[['RAIN', 'MONTH', 'Duration(Days)']] = scaler.fit_transform(X[['RAIN', 'MONTH', 'Duration(Days)']])

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
#print('X_train:', x_train.shape)
#add others





# Initialize the model
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Predict probabilities for the positive class (1)
y_proba = model.predict_proba(X_test)[:, 1]
# Calculate ROC-AUC
roc_auc = roc_auc_score(y_test, y_proba)

print("Model Evaluation Metrics:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("roc-auc:", roc_auc)


In [ ]:
#import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score

# Get the predicted probabilities for the positive class
y_proba = model.predict_proba(X_test)[:, 1]

# Calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(y_test, y_proba)

# Calculate the ROC-AUC score
roc_auc = roc_auc_score(y_test, y_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='grey', linestyle='--')  # Dashed diagonal for random guessing
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show()


In [ ]:
#Logistic Regression model

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features only on the training data
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_train_scaled[['RAIN', 'MONTH', 'Duration(Days)']] = scaler.fit_transform(X_train[['RAIN', 'MONTH', 'Duration(Days)']])

# Apply the same scaling to the test data
X_test_scaled = X_test.copy()
X_test_scaled[['RAIN', 'MONTH', 'Duration(Days)']] = scaler.transform(X_test[['RAIN', 'MONTH', 'Duration(Days)']])

# Initialize the Logistic Regression model with changes
log_reg = LogisticRegression(random_state=42,
                             max_iter=1000,  # Increased max_iter for more iterations
                             solver='liblinear',  # Use the 'liblinear' solver which works well for smaller datasets
                             C=1.0)  # Regularization strength (try adjusting if necessary)

# Train the model with scaled training data
log_reg.fit(X_train_scaled, y_train)

# Predict and evaluate on the test set
y_pred_log_reg = log_reg.predict(X_test_scaled)

# Evaluate the performance
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
precision_log_reg = precision_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg)

# Print the evaluation metrics
print("Logistic Regression Model Evaluation Metrics:")
print("Accuracy:", accuracy_log_reg)
print("Precision:", precision_log_reg)
print("Recall:", recall_log_reg)
print("F1 Score:", f1_log_reg)


In [74]:
#LSTM MODEL

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Step 1: Select features and target variable
X = merged_data_exploded.drop(columns=['Start Date', 'End Date', 'Flood Occurance'])  # Drop the irrelevant columns
y = merged_data_exploded['Flood Occurance']  # Target variable

# Step 2: Scale numerical features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 3: Convert target variable to categorical (one-hot encoding)
y_categorical = to_categorical(y)  # Convert the binary target to one-hot encoding

# Step 4: Reshape data for LSTM (LSTM expects 3D input: [samples, timesteps, features])
X_lstm = np.expand_dims(X_scaled, axis=1)  # Add a timestep dimension (1 timestep per sample)

# Step 5: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_categorical, test_size=0.2, random_state=42)

# Step 6: Build the LSTM model
model = Sequential([
    LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False),  # LSTM layer
    Dropout(0.3),  # Dropout layer for regularization
    Dense(64, activation='relu'),  # Dense hidden layer
    Dropout(0.3),  # Dropout layer for regularization
    Dense(y_categorical.shape[1], activation='softmax')  # Output layer with softmax for classification
])

# Step 7: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Step 8: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,  # Number of epochs can be adjusted
    batch_size=32,  # Batch size can be adjusted
    verbose=1
)

# Step 9: Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Step 10: Make predictions (optional)
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)  # Convert probabilities to class labels
y_test_labels = np.argmax(y_test, axis=1)  # Convert one-hot encoded y_test back to labels


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1768/1768 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.6711 - loss: 0.6050 - val_accuracy: 0.7133 - val_loss: 0.5676
Epoch 2/20
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7143 - loss: 0.5651 - val_accuracy: 0.7187 - val_loss: 0.5529
Epoch 3/20
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7229 - loss: 0.5523 - val_accuracy: 0.7231 - val_loss: 0.5467
Epoch 4/20
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.7303 - loss: 0.5411 - val_accuracy: 0.7291 - val_loss: 0.5372
Epoch 5/20
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7320 - loss: 0.5325 - val_accuracy: 0.7313 - val_loss: 0.5307
Epoch 6/20
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.7366 - loss: 0.5226 - val_accuracy: 0.7362 - val_loss: 0.5264
Epoch 7/20
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.7433 - loss: 0.5108 - val_accuracy: 0.7388 - val_loss: 0.5200
Epoch 8/20
1768/1768 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.7443 - loss: 0.5104 - val_a

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred = np.argmax(model.predict(X_test), axis=1)  # Predictions
y_true = np.argmax(y_test, axis=1)  # True labels (if one-hot encoded)

print(classification_report(y_true, y_pred))  # Precision, Recall, F1 Score
roc_auc = roc_auc_score(y_true, model.predict(X_test)[:, 1])  # ROC-AUC score
print("ROC-AUC score:", roc_auc)


In [ ]:

# If y_test is one-hot encoded, convert it to a single label (0 or 1)
if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test = np.argmax(y_test, axis=1)  # Convert one-hot to single label

# Predict probabilities for the positive class (Flood = 1)
y_pred_proba = model.predict(X_test)[:, 1]

# Compute the ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_test, y_pred_proba)

# Plot the ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f"ROC Curve (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert predicted probabilities to binary class labels
y_pred = (y_pred_proba > 0.5).astype(int)  # Threshold at 0.5 for binary classification

# Use y_test directly (already binary)
y_test_labels = y_test.astype(int)  # Ensure it's integer for comparison

# Calculate evaluation metrics
accuracy = accuracy_score(y_test_labels, y_pred)
precision = precision_score(y_test_labels, y_pred)
recall = recall_score(y_test_labels, y_pred)
f1 = f1_score(y_test_labels, y_pred)

# Print the metrics
print("LSTM Model Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
